<a href="https://colab.research.google.com/github/AndyJihang/Building-Code-Agents-with-Hugging-Face-smolagents/blob/main/Secure_Code_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Why you need secure code execution?

Code agents generate and run code. Running that code directly on your machine is risky (file/system/network damage or secret exfiltration). The core defense is isolating execution in a sandbox.

## Built-in local guards (baseline)

`smolagent`s’ Python executor blocks most imports by default; you must explicitly allow modules via additional_authorized_imports (e.g., add numpy.random, matplotlib.pyplot if you truly need them). This mitigates many obvious attacks but isn’t a full boundary.

## Two secure options
 * E2B Cloud Sandbox
 * Local containers

## Quickstart: E2B for a single agent

In [6]:
!pip install -q "smolagents[e2b,openai]"

In [7]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [8]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [13]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command
# In our interpreter, it does not.
harmful_command = "!echo Bad command"
run_capture_exception(harmful_command)

Bad command
ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [14]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['itertools', 'math', 'random', 'statistics', 'datetime', 'time', 're', 'unicodedata', 'collections', 'queue', 'stat', 'numpy']


In [15]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [16]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [12]:
# Show the allowlist
print(getattr(custom_executor, "authorized_imports", None))

['itertools', 'math', 'random', 'statistics', 'datetime', 'time', 're', 'unicodedata', 'collections', 'queue', 'stat', 'numpy']


In [19]:
# This last block (PIL spam) is intentionally not executed.
custom_executor = LocalPythonExecutor(["PIL"])
harmful_command = """
from PIL import Image
img = Image.new('RGB', (100, 100), color='blue')
i=0
while i < 10000:
    img.save(f'simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)

## Running in a sandbox

In [21]:
from google.colab import userdata

E2B_API_KEY = userdata.get("E2B_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [22]:
from smolagents import CodeAgent, OpenAIServerModel, Tool

# OpenAI model (replaces HfApiModel)
model = OpenAIServerModel(
    model_id="gpt-4o-mini",                                 # keep small/cheap; change if you like
    api_key=OPENAI_API_KEY,                   # read from env/.env
    api_base="https://api.openai.com/v1",
    temperature=0.2,
)

In [32]:
class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def _truncate(self, text: str, n: int) -> str:
        if text is None: return ""
        return text if len(text) <= n else (text[: n-3] + "...")

    def forward(self, url: str) -> str:
        try:
            import re, requests
            from markdownify import markdownify
        except ImportError as e:
            raise ImportError(
                "Please `pip install requests markdownify` in your environment."
            ) from e
        try:
            resp = requests.get(url, timeout=20)
            resp.raise_for_status()
            md = markdownify(resp.text).strip()
            md = re.sub(r"\n{3,}", "\n\n", md)
            return self._truncate(md, self.max_output_length)
        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except requests.exceptions.RequestException as e:
            return f"Error fetching the webpage: {e}"
        except Exception as e:
            return f"An unexpected error occurred: {e}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    # Allow the sandbox to install/use these plain libs if the model writes code that uses them
    additional_authorized_imports=["requests", "markdownify"],
    max_steps=1,                 # allow one self-fix step if needed
)


Initializing executor, hold on...

Requirement already satisfied: requests in /usr/local/lib/python3.12/site-packages (2.32.4)

Collecting markdownify

  Downloading markdownify-1.2.0-py3-none-any.whl.metadata (9.9 kB)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/site-packages (from requests) 
(3.4.3)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/site-packages (from requests) (3.10)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/site-packages (from requests) 
(2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/site-packages (from requests) 
(2025.8.3)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.12/site-packages (from markdownify)
(4.13.4)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.12/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.7)
Requirement already satisfied: typing-extensions>=4.0.0 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (4.14.1)

Downloading markdownify-1.2.0-py3-none-any.whl (15 kB)

Installing collected packages: markdownify

Successfully installed markdownify-1.2.0
WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager, possibly rendering your system unusable. It is recommended to use a virtual environment 
instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and 
want to suppress this warning.


E2B is running

In [34]:
prompt = (
  "Use the `visit_webpage` tool to fetch https://github.com/huggingface?tab=repositories. "
  "Do NOT print the raw page content. Extract ONE top repo (prefer a pinned repo; otherwise the highest-star repo "
  "you can see). Print exactly: name: <name> | url: <url>. Then STOP."
)
out = agent.run(prompt)
print(out)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the `visit_webpage` tool to fetch https://github.com/huggingface?tab=repositories. Do NOT print the raw     │
│ page content. Extract ONE top repo (prefer a pinned repo; otherwise the highest-star repo you can see). Print   │
│ exactly: name: <name> | url: <url>. Then STOP.                                                                  │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  page_content = visit_webpage("https://github.com/huggingface?tab=repositories")                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: None

[Step 1: Duration 2.23 seconds| Input tokens: 2,094 | Output tokens: 95]

Reached max steps.

[Step 2: Duration 0.54 seconds| Input tokens: 2,462 | Output tokens: 112]

name: transformers | url: https://github.com/huggingface/transformers
